In [189]:
import requests
import json
from sklearn.manifold import TSNE

import chardet
import re
import pandas as pd
import numpy as np
from pprint import pprint
import requests
from bixin import predict
import snownlp
import dm_pb2 as Danmaku
import sklearn

In [190]:
# 1.根据bvid请求得到cid
bvid = "BV1bc411H7Gv"
def get_cid():
    url = 'https://api.bilibili.com/x/player/pagelist?bvid=BV1bc411H7Gv'
    res = requests.get(url).text
    json_dict = json.loads(res)
    pprint(json_dict)
    return (json_dict["data"][0]["duration"], json_dict["data"][0]["cid"])
(max_time, cid) = get_cid()
max_time = max_time * 1000

print(cid)



{'code': 0,
 'data': [{'cid': 1105215233,
           'dimension': {'height': 1920, 'rotate': 0, 'width': 3840},
           'duration': 87,
           'first_frame': 'http://i0.hdslb.com/bfs/storyff/n230422a219gylliozq0r520l6kdz8f1_firsti.jpg',
           'from': 'vupload',
           'page': 1,
           'part': '【4K修复+60帧】小鬼 泰裤辣（原版）',
           'vid': '',
           'weblink': ''}],
 'message': '0',
 'ttl': 1}
1105215233


In [191]:
# 更新：新版api需要使用proto获取弹幕文件
def get_prot_dm(cid):
    url = 'https://api.bilibili.com/x/v2/dm/web/seg.so'
    params = {
        'type': 1,         # 弹幕类型
        'oid': cid,    # cid
        'segment_index': 1 # 弹幕分段
    }
    resp = requests.get(url, params)
    data = resp.content

    danmaku_seg = Danmaku.DmSegMobileReply()
    danmaku_seg.ParseFromString(data)

    return danmaku_seg.elems

danmu_proto = get_prot_dm(cid)

In [192]:
#得到弹幕 array
def to_list(prot):
    l = []
    for i in range(len(prot)):
        l.append({})
        l[i]["id"] = prot[i].id
        l[i]["progress"] = prot[i].progress
        l[i]["content"] = prot[i].content

    return l
listed_danmu = to_list(danmu_proto)
df = pd.DataFrame(listed_danmu)
mask = df['content'].str.len() > 10
df = df[~mask]
df['content'] = df['content'].str.replace('[^\w\s]', '')
mask = df['content'].str.len() < 1
df = df[~mask]
df.dropna(subset=['content'], inplace=True)
mask = df['content'].str.len() == 0
df = df[~mask]
print(df)

                       id  progress content
0     1301402202560179456     23493      哥哥
1     1301445531851347456       324    我40帧
4     1301467215505580032     52312    油光满面
5     1301468690323432448     76163    音乐骤停
8     1301487277205294592     51629    脚趾扣地
...                   ...       ...     ...
3537  1311675716198407680     75684     要来力
3538  1311677326215846144     51980   那可太好了
3539  1311681064447932160     85094     泰裤辣
3540  1311681865090216192       319    我测深圳
3541  1311681944009865984     80120    不要不要

[2746 rows x 3 columns]


In [193]:
#divide into 6 parts according to time variation
def classify_sentiment(s):
    if(s<0.166):
        return 0
    if(s<0.333):
        return 1
    if(s<0.5):
        return 2
    if(s<0.666):
        return 3
    if(s<0.833):
        return 4
    if(s<=1):
        return 5
def get_sentiment(text):
    s = snownlp.SnowNLP(text)
    return classify_sentiment(s.sentiments)
index = 0
leftop = pd.DataFrame()

for i in range(6):
    start_time = max_time * i / 6
    end_time = max_time * (i+1) / 6
    tmp = df[(df['progress'] >= start_time) & (df['progress'] <= end_time)]
    tmp = tmp['content'].value_counts()
    tmp = pd.DataFrame({'content': tmp.index, 'count': tmp.values})
    mask = tmp['count'].values <= 3
    tmp = tmp[~mask]
    tmp['emo'] = tmp['content'].apply(get_sentiment)
    tmp['label'] = index
    print(tmp)
    leftop = pd.concat([leftop, tmp], axis=0)

    index+=1

leftop.to_csv("video.csv",index=False)
counts = df['content'].value_counts()
leftop

       content  count  emo  label
0         我测深圳     30    2      0
1       给我点鸡叫声     13    1      0
2   什么都修复只会害了你     12    1      0
3        我死在深圳     10    1      0
4         我是丁真      9    2      0
5    什么都修只会害了你      9    0      0
6         我40帧      7    3      0
7          泰裤辣      6    3      0
8       给我点滋尿声      5    2      0
9         我是深圳      5    2      0
10         抬裤拉      4    2      0
11        油光满面      4    3      0
     content  count  emo  label
0        董明珠     13    2      1
1         病句     10    2      1
2        泰裤辣     10    3      1
3   如约而至地出现了     10    3      1
4       油光满面      9    3      1
5          油      6    2      1
6        泰裤啦      6    3      1
7   如约而至的出现了      5    3      1
8     我还是在这里      4    3      1
9       杰夫哈迪      4    2      1
10        好油      4    3      1
  content  count  emo  label
0     住医院     16    2      2
1    家电下乡     11    4      2
2     泰裤辣      7    3      2
3     yue      4    3      2
4     做阴乐      4    3      2


,content,count,emo,label
0,我测深圳,30,2,0
1,给我点鸡叫声,13,1,0
2,什么都修复只会害了你,12,1,0
3,我死在深圳,10,1,0
4,我是丁真,9,2,0
5,什么都修只会害了你,9,0,0
6,我40帧,7,3,0
7,泰裤辣,6,3,0
8,给我点滋尿声,5,2,0
9,我是深圳,5,2,0


In [194]:
# 2.根据cid请求弹幕，解析弹幕得到最终的数据

# def get_data(cid):
#     final_url = "https://api.bilibili.com/x/v1/dm/list.so?oid=" + str(cid)
#     final_res = requests.get(final_url)
#     final_res.encoding = chardet.detect(final_res.content)['encoding']
#     final_res = final_res.text
#     pattern = re.compile('<d.*?>(.*?)</d>')
#     match = re.compile(r'<d\s+[^>]*\bp="([^"]*)"')
#     pa = match.findall(final_res)
#
#     data = pattern.findall(final_res)
#     danmu_time = [float(item.split(',')[0]) for item in pa]
#     danmu_mode = [float(item.split(',')[1]) for item in pa]
#     danmu_size = [float(item.split(',')[2]) for item in pa]
#     danmu_color = [float(item.split(',')[3]) for item in pa]
#     danmu_abstime = [float(item.split(',')[4]) for item in pa]
#     danmu_pool = [float(item.split(',')[5]) for item in pa]
#     danmu_id = [(item.split(',')[6]) for item in pa]
#     danmu_rowid = [float(item.split(',')[7]) for item in pa]
#     print(len(data))
#     return (data, danmu_time, danmu_mode, danmu_size, danmu_color, danmu_abstime, danmu_pool, danmu_id, danmu_rowid)
#
#
# (danmu_text, danmu_time, danmu_mode, danmu_size, danmu_color, danmu_abstime, danmu_pool, danmu_id, danmu_rowid) = get_data(cid)

In [195]:
# # 分割弹幕为八个部分
# # 视频持续时间
# max_time = max(danmu_time)
# divided_danmu = []
# for i in range(8):
#     time_start = max_time * (i/8)
#     time_stop = max_time * ((i+1)/8)
#     for j in range(len(danmu_time)):
#         if time_start <= danmu_time[j] <= time_stop:
#             divided_danmu.append([i, danmu_text[j]])
# print(len(danmu_time))

In [196]:
# export csv
# arr = np.array(divided_danmu)
# df = pd.DataFrame(arr)
# df.to_csv("video.csv", header=["label", "text"])

In [197]:
def get_relate(bvid):
    # 设置请求参数
    params = {
        "bvid": bvid,  # 你想获取推荐视频的原始视频的bid
        "num": 1,  # 获取的推荐视频数量，最多为40
    }

    # 发送请求
    response = requests.get("https://api.bilibili.com/x/web-interface/archive/related", params=params)

    # 解析响应
    if response.status_code == 200:
        data = response.json()
        recommend_list = data.get("data")
        # 处理推荐视频列表数据
        return recommend_list
    else:
        print("请求失败")

relate_list = get_relate(bvid)
print(relate_list[0])

{'aid': 823813911, 'videos': 1, 'tid': 130, 'tname': '音乐综合', 'copyright': 2, 'pic': 'http://i1.hdslb.com/bfs/archive/71dc7d0fa214d6db90ccd6ddd247630169fb6cb5.jpg', 'title': '小鬼最新现场，不知道该怎么评价了…..', 'pubdate': 1679316322, 'ctime': 1679316322, 'desc': 'tiktok', 'state': 0, 'duration': 15, 'rights': {'bp': 0, 'elec': 0, 'download': 0, 'movie': 0, 'pay': 0, 'hd5': 0, 'no_reprint': 0, 'autoplay': 1, 'ugc_pay': 0, 'is_cooperation': 0, 'ugc_pay_preview': 0, 'no_background': 0, 'arc_pay': 0, 'pay_free_watch': 0}, 'owner': {'mid': 269870290, 'name': '我养黑怕', 'face': 'https://i1.hdslb.com/bfs/face/da0811f5d71f2c62a09be4a73054d37e3d7ed248.jpg'}, 'stat': {'aid': 823813911, 'view': 4472398, 'danmaku': 2649, 'reply': 4905, 'favorite': 28620, 'coin': 3679, 'share': 107793, 'now_rank': 0, 'his_rank': 0, 'like': 68816, 'dislike': 0, 'vt': 0, 'vv': 4472398}, 'dynamic': '', 'cid': 1061985317, 'dimension': {'width': 1080, 'height': 1920, 'rotate': 0}, 'short_link_v2': 'https://b23.tv/BV1Hg4y1x77p', 'up_from_

In [ ]:
#迭代获取其他视频的推荐
index = 0
whole_related_df = pd.DataFrame()
for it in relate_list:
    danmu_relate = get_prot_dm(it['cid'])
    #preprocess
    listed_danmu = to_list(danmu_relate)
    df = pd.DataFrame(listed_danmu)
    mask = df['content'].str.len() > 10
    df = df[~mask]
    df['content'] = df['content'].str.replace('[^\w\s]', '')
    df['content'] = df['content'].str.replace('哈', '')
    df['content'] = df['content'].str.replace('嘿', '')
    df['content'] = df['content'].str.replace('啊', '')
    df['content'] = df['content'].str.replace('6', '')
    mask = df['content'].str.len() <= 1
    df = df[~mask]
    df.dropna(subset=['content'], inplace=True)
    #end of preprocess
    relate_tmp = df['content'].value_counts()
    relate_tmp = pd.DataFrame({'content': relate_tmp.index, 'count': relate_tmp.values})
    relate_tmp['emo'] = relate_tmp['content'].apply(get_sentiment)
    relate_tmp.to_csv("relate_video_"+str(index)+".csv")
    whole_related_df = pd.concat([df, whole_related_df], axis=0)

    index+=1

whole_related_df['id'] /= 100000000
whole_related_df

In [ ]:
# whole_count = whole_related_df['content'].value_counts()
# whole_count = pd.DataFrame({'content': whole_count.index, 'count': whole_count.values})
whole_count = whole_related_df.groupby('content').agg({'id': 'sum', 'content': 'size'}).rename(columns={'content': 'count'}).reset_index()
mask = whole_count['count'].values <= 5
whole_count = whole_count[~mask]
whole_count
whole_count['emo'] = whole_count['content'].apply(get_sentiment)

In [ ]:
whole_count.to_csv("main.csv")
print(whole_count)
whole_count

In [ ]:
# 降维 用于主图
# 使用t-SNE进行降维
mask = whole_count['content'].str.len() <= 1
whole_count = whole_count[~mask]
tsne = TSNE(n_components=2, random_state=42)
embedding = tsne.fit_transform(whole_count.drop('content', axis=1))

# 创建降维后的DataFrame
df_tsne = pd.DataFrame(embedding, columns=['Dimension 1', 'Dimension 2'])
whole_count.index = df_tsne.index
df_tsne['content'] = whole_count['content']
df_tsne['count'] = whole_count['count']
df_tsne.to_csv("tsne.csv")
df_tsne


In [ ]:
#河流图
index = 0
river = pd.DataFrame()
for i in range(10):
    start_time = max_time * i / 10
    end_time = max_time * (i+1) / 10
    tmp = whole_related_df[(whole_related_df['progress'] >= start_time) & (whole_related_df['progress'] <= end_time)]
    tmp = tmp['content'].value_counts()
    tmp = pd.DataFrame({'content': tmp.index, 'count': tmp.values})
    mask = tmp['count'].values <= 3
    tmp['emo'] = tmp['content'].apply(get_sentiment)
    tmp = tmp['emo'].value_counts()
    tmp = tmp.sort_index()
    tmp = pd.DataFrame(tmp)
    tmp = tmp.T
    river = pd.concat([river, tmp], axis=0)
river.to_csv()


In [ ]:
# # 3.保存弹幕列表
#
# def save_to_file(data):
#     with open("dan_mu.txt", mode="w", encoding="utf-8") as f:
#         for i in data:
#             f.write(i)
#             f.write("\n")
# save_to_file(data)


In [ ]:
# # 1 导入相关库
# import pandas as pd
# import jieba
# from wordcloud import WordCloud
# import matplotlib.pyplot as plt
# from imageio import imread

In [ ]:
# # 目标视频文件的处理
# df = pd.read_csv("video.csv")
# array = np.array(df)
# content = array[:, 3]
# progress = array

In [ ]:

# import warnings
# warnings.filterwarnings("ignore")
#
# # 2 读取文本文件，并使用lcut()方法进行分词
# with open("video.csv",encoding="utf-8") as f:
#     txt = f.read()
# txt = txt.split()
# data_cut = [jieba.lcut(x) for x in txt]
# data_cut
# # 3 读取停用词
# stop = [" ","道","说道","说",'了']
# # 4 去掉停用词之后的最终词
# s_data_cut = pd.Series(data_cut)
# all_words_after = s_data_cut.apply(lambda x:[i for i in x if i not in stop])
# # 5 词频统计
# all_words = []
# for i in all_words_after:
#     all_words.extend(i)
# word_count = pd.Series(all_words).value_counts()
# # 6 词云图的绘制
# # 1）读取背景图片
#
#
# # 2）设置词云参数
# wc = WordCloud(font_path="C:/Users/Windows/fonts/simhei.ttf",
#                background_color="white",
#                max_words=2000,
#                max_font_size=200,
#                random_state=42
#               )
# wc2 = wc.fit_words(word_count)
#
# # 3）绘制词云图
# plt.figure(figsize=(16,8), dpi=300)
# plt.imshow(wc2)
# plt.axis("off")
# plt.show()
# wc.to_file("ciyun.jpg")

